# Code Documenter
This application documents a code module:
Generates a DocString
Inserts comments where applicable

Output is in TextBox and can be written to a file.

Offeres model selection

In [ ]:
# Import Libraries

from dotenv import load_dotenv
import os
#import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import anthropic
from google import genai
from google.genai import types
# from google.colab import drive
from huggingface_hub import login
#from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, TextIteratorStreamer
import torch
import gradio as gr
#import threading 

In [ ]:
# Constants - model names
LLAMA_MODEL = "codellama-7b-kstack" # we will be using LM_Studio for this model
QWEN_MODEL = "qwen2.5-coder-14b-instruct" # we will be using LM_Studio for this model, might be too large for some systems
OPENAI_MODEL = "gpt-4o"
ANTHROPIC_MODEL = "claude-3-5-haiku-latest"
GOOGLE_MODEL = "gemini-2.5-pro"
model_choices = [LLAMA_MODEL, QWEN_MODEL, OPENAI_MODEL, ANTHROPIC_MODEL, GOOGLE_MODEL]

In [ ]:
# Load environment variables and set up API connections
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
hf_api_key = os.getenv('HF_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
    openai = OpenAI(api_key=openai_api_key)
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
    claude = anthropic.Anthropic(api_key=anthropic_api_key)
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
    gemini =  genai.Client(api_key=google_api_key)  
else:
    print("Google API Key not set")

if hf_api_key:
    print(f"HuggingFace API Key exists and begins {hf_api_key[:7]}")
    login(hf_api_key, add_to_git_credential=True)
else:
    print("HuggingFace API Key not set")

# Set up LM Studio connection
lm_studio_via_openai = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="lmstudio")

In [ ]:
# Model invocation function
# This function will handle the invocation of different models based on the selected model name.

def invoke_model(model_name, prompt, max_tokens=1000, temperature=0.4):
    if model_name == OPENAI_MODEL:
        stream = openai.chat.completions.create(
            model=OPENAI_MODEL,
            messages=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            stream=True
        )
        #return response.choices[0].message.content
        for chunk in stream:
            yield chunk.choices[0].delta.content or ''
    
    elif model_name == ANTHROPIC_MODEL:
        #Invoke Claude model
        # Handle the prompt structure for Claude
        #print(f"Invoking model: {model_name}")
        #print(f"System prompt: {prompt[0]['content']}")
        #print(f"User prompt: %s", prompt[1]['content'][:100])
        try:
            # Use context manager for proper stream handling
            with claude.messages.stream(
                model=ANTHROPIC_MODEL,
                system=prompt[0]['content'],
                messages=[prompt[1]],
                max_tokens=max_tokens,
                temperature=temperature
            ) as stream:
                #print("Stream created successfully")
                chunk_count = 0
                
                for chunk in stream.text_stream:
                    chunk_count += 1
                    #print(f"Chunk {chunk_count}: {repr(chunk)}")  # Use repr to see None/empty values
                    
                    if chunk:  # Only yield non-empty chunks
                        yield chunk
                        
            #print(f"Stream completed. Total chunks: {chunk_count}")
            
        except Exception as e:
            print(f"Error invoking Claude model: {e}")
            yield f"Error invoking Claude model: {e}"
            return
    
    elif model_name == GOOGLE_MODEL:
        #Invoke Gemini model
        # Handle the prompt structure for Gemini
        #print(f"Invoking model: {model_name}")
        stream = gemini.models.generate_content_stream(
            model=GOOGLE_MODEL,
            contents=prompt[1]['content'],
            config=types.GenerateContentConfig(
                temperature=temperature,
                maxOutputTokens=max_tokens,
                system_instruction=prompt[0]['content'],)
        )
        #print("Streaming response from Gemini...")
        for chunk in stream:
            yield chunk.text or ''
    
    elif model_name == LLAMA_MODEL or model_name == QWEN_MODEL:
        # invoke LM Studio model
        #print(f"Invoking model: {model_name}")
        stream = lm_studio_via_openai.chat.completions.create(
            model=model_name,
            messages=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            stream=True
        )
        #print("Streaming response from LM Studio...")
        #return response.choices[0].message.content
        #response=""
        for chunk in stream:
            #response += chunk.choices[0].delta.content or ''
            yield chunk.choices[0].delta.content or ''
    else:
        raise ValueError("Unsupported model name")

In [ ]:
# Save text to a selected location
# This function will save the generated text to a specified file or the current directory if no file
def save_text_to_selected_location(text_content):
    if not text_content.strip():
        return "No content to save"

    save_path = "output.txt"

    # If no file is selected, save to current directory
    try:
        with open(save_path, 'w', encoding='utf-8') as f:
            f.write(text_content)
        return f"Successfully saved to: {save_path}"
    except Exception as e:
        return f"Error saving file: {str(e)}"
    
# Set up event handlers
def generate_response(system_input, prompt, max_tokens, temperature, model_name):
    if system_input == "Documentation":
        system_prompt = """You are an experienced coding assistant. You will identify the programming language used in a provided code snippet and generate documentation for the code. 
        Ensure generally acceptable documentation standards are followed. Also generate short inline comments where applicable to explain complicated code. Respond ONLY with the updated code 
        with documentation and comments, do not include any other preamble or explanation."""
    elif system_input == "Test Code Generation":
        system_prompt = """You are an experienced coding assistant. You will identify the programming language used in a provided code function and generate test code for it. " \
        "The code should test against normal and edge cases, and ensure proper error handling:"""
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": f"This is the code to process: ```\n{prompt}\n```"}]
    try:
        acumulated_response = ""
        for chunk in invoke_model(model_name=model_name, prompt=messages, max_tokens=max_tokens, temperature=temperature):
            acumulated_response += chunk
            yield acumulated_response
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("# Coding Assistant\n Choose from the available models to generate responses. Choose from either Documentation generation, or Test Code generation.")
    with gr.Row():
        system_input = gr.Dropdown(label="Task Type", choices=["Documentation", "Test Code Generation"], value="Documentation", interactive=True, visible=True)
    with gr.Row():
        prompt_input= gr.Textbox(label="Enter your prompt", placeholder="Type your prompt here...", lines=4)
        response_output = gr.Textbox(label="Model Response", lines=10, interactive=False)
    with gr.Row():
        max_tokens_input = gr.Slider(minimum=1, maximum=4096, value=1000, step=1, label="Max Tokens")
        temperature_input = gr.Slider(minimum=0.0, maximum=1.0, value=0.7, step=0.05, label="Temperature")
        model_selector = gr.Dropdown(
            label="Select Model", 
            choices=model_choices, 
            value=LLAMA_MODEL, 
            interactive=True
            )
    with gr.Row():
        generate_button = gr.Button("Generate Response", visible=True)
        download_button = gr.Button("Download Response", visible=True)
    
    generate_button.click(
        fn=generate_response,
        inputs=[system_input, prompt_input, max_tokens_input, temperature_input, model_selector],
        outputs=response_output
    )
    download_button.click(
        fn=save_text_to_selected_location,
        inputs=[response_output],
        outputs=None
    )

# Launch the UI
ui.launch(inbrowser=True)